In [73]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation, preprocessing, metrics

In [74]:
df_train = pd.read_csv('aggregated_train_with_nummosquitos.csv')# from make_aggregated_with_Nummosquitos
#df_test = pd.read_csv('../test_filled_new.csv')
#orig = pd.read_csv('../test_filled_new.csv')
df_test = pd.read_csv('../test_filled_new.csv')

In [75]:
#df_train.columns,df_test.columns

In [76]:
'Species' in df_test.columns

True

In [77]:
[var for var in df_train.columns if var not in df_test.columns]

['Unnamed: 0', 'NumMosquitos', 'WnvPresent']

In [78]:
X = df_train.copy()
X.dtypes

Unnamed: 0                  int64
Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
Station                     int64
Tmax                        int64
...
16_week_avrgTavg           float64
17_week_avrgPrecipTotal    float64
17_week_avrgTavg           float64
18_week_avrgPrecipTotal    float64
18_week_avrgTavg           float64
19_week_avrgPrecipTotal    float64
19_week_avrgTavg           float64
20_week_avrgPrecipTotal    float64
20_week_avrgTavg           float64
21_week_avrgPrecipTotal    float64
21_week_avrgTavg           float64
22_week_avrgPrecipTotal    float64
22_week_avrgTavg           float64
23_week_avrgPrecipTotal    floa

In [79]:
[var for var in df_test.columns if var not in X.columns]

['Id']

In [80]:
df_test.Species.values

array(['CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS', 'CULEX PIPIENS', ...,
       'CULEX TARSALIS', 'UNSPECIFIED CULEX', 'CULEX ERRATICUS'], dtype=object)

In [81]:
string_features = ['Date','Address','Street','AddressNumberAndStreet','CodeSum']
species_encoder = preprocessing.LabelEncoder()
trap_encoder = preprocessing.LabelEncoder()
#X_dump = X[string_features]
df_dump = df_test[string_features]
#X_dump = X[string_features]

df_test.Species[df_test.Species == 'UNSPECIFIED CULEX'] = 'CULEX PIPIENS/RESTUANS'
X.Species = species_encoder.fit_transform(X.Species)
y = df_train['NumMosquitos']
df_test.Species = species_encoder.transform(df_test.Species)
df_train.Species = species_encoder.transform(df_train.Species)


/home/thomas/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [82]:

df_test.Trap = trap_encoder.fit_transform(df_test.Trap)
X.Trap = trap_encoder.transform(X.Trap)
df_train.Trap = trap_encoder.transform(df_train.Trap)

try:
 X = X.drop(string_features,axis=1)
except:
    pass
try:
 df_test = df_test.drop(string_features,axis=1)
except:
    pass
try:
 df_test = df_test.drop('Id',axis=1)
except: 
    pass
#X.head()
try:
    X = X.drop(['Unnamed: 0', 'NumMosquitos', 'WnvPresent'],axis=1)
except:
        pass

In [83]:
df_dump.head()

,Date,Address,Street,AddressNumberAndStreet,CodeSum
0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",
1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",
2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",
3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",
4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",


In [84]:
X.dtypes[:70]

Species              int64
Block                int64
Trap                 int64
Latitude           float64
Longitude          float64
AddressAccuracy      int64
Station              int64
Tmax                 int64
Tmin                 int64
Tavg               float64
Depart             float64
DewPoint             int64
WetBulb            float64
Heat                 int64
Cool                 int64
...
13_week_avrgPrecipTotal    float64
13_week_avrgTavg           float64
14_week_avrgPrecipTotal    float64
14_week_avrgTavg           float64
15_week_avrgPrecipTotal    float64
15_week_avrgTavg           float64
16_week_avrgPrecipTotal    float64
16_week_avrgTavg           float64
17_week_avrgPrecipTotal    float64
17_week_avrgTavg           float64
18_week_avrgPrecipTotal    float64
18_week_avrgTavg           float64
19_week_avrgPrecipTotal    float64
19_week_avrgTavg           float64
20_week_avrgPrecipTotal    float64
Length: 70, dtype: object

In [85]:
[var for var in X.columns if var not in df_test.columns]

[]

In [86]:
[var for var in df_test.columns if var not in X.columns]

[]

In [87]:
clf = RandomForestRegressor(n_estimators=1000,n_jobs=-1)

In [88]:
#X = X[:20]
#y = y[:20]

In [89]:
from sklearn import gaussian_process
#clf = gaussian_process.GaussianProcess()

In [90]:
clf.fit(X,y)

RandomForestRegressor(bootstrap=True, compute_importances=None,
           criterion='mse', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
           min_samples_split=2, n_estimators=1000, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0)

In [91]:
yhat = clf.predict(df_test)

##ToDO: Train different regressors (min 5), mean them and use std of the values as margin of error

In [92]:
y.mean(),yhat.mean()

(12.853512278697886, 11.494908109113737)

In [93]:
clf.score(df_test,yhat)

1.0

In [94]:
df_test['NumMosquitos'] = yhat

In [95]:
len(df_test)

116293

In [96]:
df_test = pd.concat([df_test,df_dump],axis=1)

In [97]:
df_test.to_csv('aggregated_test_NumMosquito_no_strings.csv',index_label = 'Id')

In [98]:
[var for var in df_test.columns if var not in df_train.columns]

[]

In [99]:
[var for var in df_train.columns if var not in df_test.columns]

['Unnamed: 0', 'WnvPresent']

In [100]:
df_train = df_train.drop('Unnamed: 0',axis=1)

In [101]:
[var for var in df_train.columns if var not in df_test.columns]

['WnvPresent']

In [102]:
df_train.to_csv('aggregated_train_NumMosquito_no_strings.csv',index_label = 'Id')